In [1]:
import os

os.environ["GROQ_API_KEY"] = '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'

In [2]:
import pickle
pickle_file = "/Users/hadiibrahim/Dev/prima-power-hmi-assistant/src/docs.pkl"
with open(pickle_file, "rb") as file:
    docs = pickle.load(file)

In [5]:
import time
from langchain_groq import ChatGroq
from langchain.callbacks.base import BaseCallbackHandler

class RateLimitCallback(BaseCallbackHandler):
    def __init__(self, llm_name, limit=29, wait_time=5):
        self.llm_name = llm_name
        self.request_count = 0
        self.limit = limit
        self.wait_time = wait_time

    def on_llm_end(self, response, **kwargs):
        self.request_count += 1
        print(f"LLM: {self.llm_name} | Request Count: {self.request_count}")
        if self.llm_name == 'mixtral-8x7b-32768' and  self.request_count % self.limit == 0 and self.request_count < 3341:
            print(f"Reached {self.limit} requests for {self.llm_name}. Waiting for {self.wait_time} seconds.")
            time.sleep(self.wait_time)
        elif self.llm_name == 'mixtral-8x7b-32768' and  self.request_count % 29 == 0 and self.request_count >= 3341:
            print(f"Reached 25 requests for {self.llm_name}. Waiting for 2 seconds.")
            time.sleep(2)
        if self.llm_name == 'llama3-70b-819' and  self.request_count % 29 == 0:
            print(f"Reached 25 requests for {self.llm_name}. Waiting for 2 seconds.")
            time.sleep(2)

# Create instances of the custom callback with the LLM name
rate_limit_callback_mixtral = RateLimitCallback(llm_name="mixtral-8x7b-32768", limit=1, wait_time=30)
rate_limit_callback_llama = RateLimitCallback(llm_name="llama3-70b-8192", limit=1, wait_time=30)


In [6]:
from langchain_groq import ChatGroq

generator_llm = ChatGroq(model_name="mixtral-8x7b-32768", 
                         streaming=False,  # Disable if not necessary
                         callbacks=[rate_limit_callback_mixtral]
                         )
critic_llm = ChatGroq(model_name="llama3-70b-8192", 
                         streaming=False,  # Disable if not necessary
                         callbacks=[rate_limit_callback_llama]
                         )


In [7]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings, HuggingFaceEmbeddings

embeddings = HuggingFaceBgeEmbeddings(
                model_name="BAAI/bge-large-en-v1.5", # or sentence-trainsformers/all-MiniLM-L6-v2
                model_kwargs={"device":"cpu"},
                encode_kwargs={"normalize_embeddings":True}
            )

/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/hadiibrahim/anaconda3/envs/temp-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and wil

In [8]:
import time
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
import groq

# Initialize the generator
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

# Generate testset with retries
testset = generator.generate_with_langchain_docs(docs, 50, distributions)
print(testset)


embedding nodes:  27%|██▋       | 1777/6648 [44:35<2:44:55,  2.03s/it]

LLM: mixtral-8x7b-32768 | Request Count: 893
Reached 1 requests for mixtral-8x7b-32768. Waiting for 30 seconds.


embedding nodes:  27%|██▋       | 1778/6648 [44:35<2:06:33,  1.56s/it]

LLM: mixtral-8x7b-32768 | Request Count: 894
Reached 1 requests for mixtral-8x7b-32768. Waiting for 30 seconds.


embedding nodes:  27%|██▋       | 1779/6648 [44:37<2:11:41,  1.62s/it]

...

embedding nodes:  42%|████▏     | 2814/6648 [1:19:17<1:20:40,  1.26s/it]

LLM: mixtral-8x7b-32768 | Request Count: 1413
Reached 1 requests for mixtral-8x7b-32768. Waiting for 30 seconds.


embedding nodes:  42%|████▏     | 2815/6648 [1:19:19<1:31:48,  1.44s/it]

LLM: mixtral-8x7b-32768 | Request Count: 1414
Reached 1 requests for mixtral-8x7b-32768. Waiting for 30 seconds.


embedding nodes:  42%|████▏     | 2816/6648 [1:19:19<1:16:37,  1.20s/it]